## Activate SnooPi Line Follower

#### Import Libraries and log info

In [ ]:
#!/usr/bin/python3
#coding=utf8
import sys
sys.path.append('/home/pi/work/notebooks') # custom version of TurboPI
import cv2
import time
import math
import random
import signal
import threading
import board
import neopixel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
from PIL import Image
from time import sleep
from datetime import datetime
from sys import exit
from datetime import datetime
from sys import exit
import subprocess
from gpiozero import LED, Buzzer, Button, MotionSensor
import signal
from signal import pause
from smbus2 import SMBus, i2c_msg

import FCSRSDK.yaml_handle as yaml_handle
import FCSRSDK.Camera as Camera
import FCSRSDK.FCSRSonar as Sonar
import FCSRSDK.FCSRBoard as Board
import FCSRSDK.FCSRMecanumChassis as mecanum
import FCSRSDK.FSCRFourInfrared as infrared

if sys.version_info.major == 2:
    print('Please run this program with python3!')
    sys.exit(0)

print('''
**********************************************************
******** BUSH FCSR Robot Line Follower Test **********
**********************************************************
----------------------------------------------------------
FCSR website:https://chandrunarayan.github.io/fcsr
----------------------------------------------------------
Tips:
 * Press ctrl-c or stop button to exit
----------------------------------------------------------
''')  

#### Line Follower Functions Block

In [ ]:
#
# Line Follower Functions Library
#
def load_config():
    global lab_data, servo_data
    
    lab_data = yaml_handle.get_yaml_data(yaml_handle.lab_file_path)
    servo_data = yaml_handle.get_yaml_data(yaml_handle.servo_file_path)

def initMove():
    car.set_velocity(0,90,0)
    Board.setPWMServoPulse(1, servo1, 1000)
    Board.setPWMServoPulse(2, servo2, 1000)

def setBuzzer(timer):
    Board.setBuzzer(0)
    Board.setBuzzer(1)
    time.sleep(timer)
    Board.setBuzzer(0)


def reset(): 
    global car_stop
    global color_list
    global detect_color
    global start_pick_up
    global servo1, servo2
    
    car_stop = False
    color_list = []
    detect_color = 'None'
    servo1 = servo_data['servo1']
    servo2 = servo_data['servo2']

def init():
    print("LineFollower Init")
    load_config()
    reset()
    initMove()

def start():
    global debug
    global __isRunning
    reset()
    __isRunning = True
    speed = 60
    car.set_velocity(speed,90,0,debug) 
    if debug: 
        print(f'debug car.set_velocity({speed},90,0)') #reset
    print("LineFollower Start")

def stop():
    global car_stop
    global debug
    global __isRunning
    car_stop = True
    __isRunning = False
    set_rgb('None')
    print("LineFollower Stop")

def exit():
    global car_stop
    global debug
    global __isRunning
    car_stop = True
    __isRunning = False
    set_rgb('None')
    print("LineFollower Exit")

def setTargetColor(color):
    global target_color

    target_color = color
    return (True, ())


def set_rgb(color):
    global num_pixels, pixels
    global rd, gr, bl, cy, mg, ye, wh, of
    Board.pixel_colors()
    if color == "red":
        pixels.fill(rd)
        pixels.show()
    elif color == "green":
        pixels.fill(gr)
        pixels.show()
    elif color == "blue":
        pixels.fill(bl)
        pixels.show()
    else:
        pixels.fill(of)
        pixels.show()

def getAreaMaxContour(contours):
    contour_area_temp = 0
    contour_area_max = 0
    area_max_contour = None

    for c in contours:  
        contour_area_temp = math.fabs(cv2.contourArea(c))  
        if contour_area_temp > contour_area_max:
            contour_area_max = contour_area_temp
            if contour_area_temp > 300:  
                area_max_contour = c

    return area_max_contour, contour_area_max  

def move():
    global car_stop
    global debug
    global __isRunning
    global detect_color
    global rd, gr, bl, cy, mg, ye, wh, of

    while True:
        if __isRunning:
            if detect_color != 'red':
                set_rgb(detect_color) 
                sensor_data = line.readData() 

                if not sensor_data[0] and sensor_data[1] and sensor_data[2] and not sensor_data[3]:
                    speed = 60
                    car.set_velocity(speed,90,0,debug) 
                    if debug: 
                        print(f'debug car.set_velocity({speed},90,0)') #reset
                    car_stop = True

                elif not sensor_data[0] and not sensor_data[1] and sensor_data[2] and not sensor_data[3]:
                    speed = 60
                    car.set_velocity(speed,90,0.03,debug) 
                    if debug:
                        print(f'debug car.set_velocity({speed},90,0.03)') #reset
                    car_stop = True

                elif not sensor_data[0] and  sensor_data[1] and not sensor_data[2] and not sensor_data[3]:
                    speed = 60
                    car.set_velocity(speed,90,-0.03,debug) 
                    if ndebug: 
                        print(f'debug car.set_velocity({speed},90,-0.03)') #reset
                    car_stop = True

                elif not sensor_data[0] and not sensor_data[1] and not sensor_data[2] and sensor_data[3]:
                    speed = 60
                    car.set_velocity(speed,90,0.03,debug) 
                    if debug:
                        print(f'debug car.set_velocity({speed},90,0.03)') #reset
                    car_stop = True

                elif sensor_data[0] and not sensor_data[1] and not sensor_data[2] and not sensor_data[3]:
                    speed = 60
                    car.set_velocity(speed,90,-0.03,debug) 
                    if debug:
                        print(f'debug car.set_velocity({speed},90,-0.03)') #reset
                    car_stop = True
                
                elif sensor_data[0] and sensor_data[1] and sensor_data[2] and sensor_data[3]:
                    if car_stop:
                        speed = 0
                        car.set_velocity(speed,90,0,debug) 
                        if debug:
                            print(f'debug car.set_velocity({speed},90,0)') #reset
                        car_stop = False
                    time.sleep(0.01)
                    
                if detect_color == 'green':
                    if not car_stop:
                        speed = 60
                        car.set_velocity(speed,90,0,debug) 
                        if debug:
                            print(f'debug car.set_velocity({speed},90,0)') #reset
                        car_stop = True

            else:  
                if car_stop:
                    setBuzzer(0.1)
                    set_rgb(detect_color) 
                    speed = 0
                    car.set_velocity(speed,90,0,debug) 
                    if debug:
                        print(f'debug car.set_velocity({speed},90,0)') #reset
                    car_stop = False
                time.sleep(0.01)
                    
        else:
            if car_stop:
                speed = 0
                car.set_velocity(speed,90,0,debug)
                if debug:
                    print(f'debug car.set_velocity({speed},90,0)') #reset
                car_stop = False
            time.sleep(0.01)


def run(img):
    global debug
    global __isRunning
    global detect_color, draw_color, color_list
    
    if not __isRunning:  
        return img
    
    img_copy = img.copy()
    img_h, img_w = img.shape[:2]
    
    frame_resize = cv2.resize(img_copy, size, interpolation=cv2.INTER_NEAREST)
    frame_gb = cv2.GaussianBlur(frame_resize, (3, 3), 3)
    frame_lab = cv2.cvtColor(frame_gb, cv2.COLOR_BGR2LAB)  

    max_area = 0
    color_area_max = None
    areaMaxContour_max = 0
    for i in target_color:
        if i in lab_data:
            frame_mask = cv2.inRange(frame_lab,
                                         (lab_data[i]['min'][0],
                                          lab_data[i]['min'][1],
                                          lab_data[i]['min'][2]),
                                         (lab_data[i]['max'][0],
                                          lab_data[i]['max'][1],
                                          lab_data[i]['max'][2]))  
            opened = cv2.morphologyEx(frame_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))  
            closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))  
            contours = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]  
            areaMaxContour, area_max = getAreaMaxContour(contours)  
            if areaMaxContour is not None:
                if area_max > max_area:  
                    max_area = area_max
                    color_area_max = i
                    areaMaxContour_max = areaMaxContour
    if max_area > 2500:  
        rect = cv2.minAreaRect(areaMaxContour_max)
        box = np.int0(cv2.boxPoints(rect))
        cv2.drawContours(img, [box], -1, range_rgb[color_area_max], 2)
        
        if color_area_max == 'red':  
            color = 1
        elif color_area_max == 'green':  
            color = 2
        else:
            color = 0
        color_list.append(color)
        
        if len(color_list) == 3:  
            color = np.mean(np.array(color_list))
            color_list = []
            start_pick_up = True
            if color == 1:
                detect_color = 'red'
                draw_color = range_rgb["red"]
            elif color == 2:
                detect_color = 'green'
                draw_color = range_rgb["green"]
            else:
                detect_color = 'None'
                draw_color = range_rgb["black"]
    else:
        detect_color = 'None'
        draw_color = range_rgb["black"]
                   
    cv2.putText(img, "Color: " + detect_color, (10, img.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, draw_color, 2) 
    
    return img

def encode_img_to_jpg(frame):
    frame_img = frame.copy()   
    frame_resize = cv2.resize(frame_img, (640, 480)) 
    frame_rotate = cv2.rotate(frame_resize, cv2.ROTATE_90_COUNTERCLOCKWISE)
    #cv2.imshow('frame', frame_resize)_,result = cv2.imencode('.jpg', frame)
    _,frame_jpg = cv2.imencode('.jpg', frame_rotate)
    return frame_jpg

    
def pixel_colors(): # Setting up color arrays for pixels
    ### setup a color array of RGB to say rd, gr, bl, cy, mg, ye, wh, of
    global rd, gr, bl, cy, mg, ye, wh, of
    of = [0,0,0]
    wh = [255,255,255]
    
    #. Modify to setup all the colors below
    rd = [255,0,0]
    gr = [0,255,0]
    bl = [0,0,255]
    cy = [0,255,255]
    mg = [255,0,255]
    ye = [255,255,0]
    of = [0,0,0]

def reset():       # simply set leds off
    global num_pixels, pixels
    global rd, gr, bl, cy, mg, ye, wh, of
    pixel_colors()
    for ii in range(num_pixels):
        pixels[ii]=of
    pixels.show()
    sleep(0.1)

def initialize():   # set leds white-off
    global num_pixels, pixels
    global rd, gr, bl, cy, mg, ye, wh, of
    pixel_colors()
    for ii in range(3):         # flash white 3-times
        pixels.fill(of)
        #print(pixels)
        pixels.show()
        sleep(0.3)
        pixels.fill(wh)
        #print(pixels)
        pixels.show()
        sleep(0.3)
    pixels.fill(of)
    #print(pixels)
    pixels.show()    

def finish():
    global num_pixels, pixels
    global rd, gr, bl, cy, mg, ye, wh, of
    pixel_colors()
    for ii in range(6):         # flash cyan 3-times
        pixels.fill(of)
        #print(pixels)
        pixels.show()
        sleep(1)
        if ii == 0:
            cl = rd
        if ii == 1:
            cl = gr
        if ii == 2:
            cl = bl            
        if ii == 3:
            cl = cy
        if ii == 4:
            cl = mg
        if ii == 5:
            cl = ye 
        pixels.fill(cl)
        #print(pixels)
        pixels.show()
        sleep(0.3)    
    pixels.fill(of)
    #print(pixels)
    pixels.show()        
    

#### Functions & Globals Block

In [ ]:

### setup neopixel board below
###
pixel_pin = board.D18  # GPIO 18

num_pixels = 2   # 2 neopixels in string

ORDER = neopixel.GRB  # Order of neopixel color Green-Red-Blue

pixels = neopixel.NeoPixel(pixel_pin, num_pixels, 
                           brightness=0.1, auto_write=False,
                           pixel_order=ORDER)

Board.reset() # turn off LEDs

servo1 = 1500
servo2 = 1500
car_stop = False
color_list = []
size = (640, 480)
detect_color = 'None'
target_color = ('red', 'green')

of = [0,0,0]
wh = [255,255,255]

#. Modify to setup all the colors below
rd = [255,0,0]
gr = [0,255,0]
bl = [0,0,255]
cy = [0,255,255]
mg = [255,0,255]
ye = [255,255,0]
of = [0,0,0]

lab_data = None
servo_data = None

range_rgb = {
    'red': (0, 0, 255),
    'blue': (255, 0, 0),
    'green': (0, 255, 0),
    'black': (0, 0, 0),
    'white': (255, 255, 255),
}

draw_color = range_rgb["black"]

car = mecanum.MecanumChassis()
line = infrared.FourInfrared()

def add_dist_to_img(img):
    global debug
    global HWSONAR
    global distance
    global distance_data
    TextSize = 12
    TextColor = (0, 255, 255)
    
    dist = HWSONAR.getDistance() / 10.0 

    distance_data.append(dist) 
    data = pd.DataFrame(distance_data)
    data_ = data.copy()
    u = data_.mean()  
    std = data_.std()  

    data_c = data[np.abs(data - u) <= std]
    distance = data_c.mean()[0]

    if len(distance_data) == 5: 
        distance_data.remove(distance_data[0])

    return cv2.putText(img, "Dist:%.1fcm"%distance, (30, 480-30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, TextColor, 2)

def encode_img_to_jpg(frame):
    frame_img = frame.copy() 
    frame_dist = add_dist_to_img(frame_img)  
    frame_resize = cv2.resize(frame_dist, (640, 480)) 
    frame_rotate = cv2.rotate(frame_resize, cv2.ROTATE_90_COUNTERCLOCKWISE)
    #cv2.imshow('frame', frame_resize)_,result = cv2.imencode('.jpg', frame)
    _,frame_jpg = cv2.imencode('.jpg', frame_rotate)
    return frame_jpg

def manual_stop(signum, frame):
    global debug
    global __isRunning
    
    print('Stopping...')
    __isRunning = False
    car.set_velocity(0,90,0,debug) 
    if debug: 
        print("debug car.set_velocity(0,90,0)") #reset
    print('Stopped')

def showVideo(VideoIndex=0, scale=1.5):
    global show
    
    try:
        cap = cv2.VideoCapture(VideoIndex)
    except:
        print("Cannot Open Device")
    try:
        ret, frame = cap.read()
        
        while(ret==True):
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if not ret:
                # Release the Video Device if ret is false
                cap.release()
                # Message to be displayed after releasing the device
                print ("Released Video Resource")
                break
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if show:
                half_frame = cv2.resize(frame, (0, 0), fx = scale, fy = scale)
                # https://medium.com/@kostal91/displaying-real-time-webcam-stream-in-ipython-at-relatively-high-framerate-8e67428ac522
                _,ret_array = cv2.imencode('.jpg', half_frame)
                i = display.Image(data=ret_array)
                display.display(i)
                display.clear_output(wait=True)
            
    except KeyboardInterrupt:
        global debug
        global __isRunning
        print('Stopping car...')
        __isRunning = False
        car.set_velocity(0,90,0,debug) 
        if debug: 
            print("debug car.set_velocity(0,90,0)") #reset
        print('Stopped')
        # Release the Video Device
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource from KeyboardInterrupt")
    pass

    

#### MAIN Block

In [ ]:
__isRunning = True

debug = False  # Do not move SnooPi if True
show = False  # Do not show images if False

th = threading.Thread(target=move)
th.setDaemon(True)
th.start()
if __name__ == '__main__':
    print("debugging - SnooPi movement is suppressed") if debug else print("warning - SnooPi might move suddenly!")
    print("SnooPi video is ON!") if show else print("SnooPi video is OFF")
    init()
    start()
    
    while __isRunning:
        showVideo(0, 1.5)
